1. 從 Notion 獲取資料並轉換成 DataFrame

In [4]:
import requests
import pandas as pd
from notion_client import Client

NOTION_API_KEY = "your_notion_api_key"
PAGE_ID = "your_page_id"
HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}

def get_notion_page_content(page_id):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(url, headers=HEADERS)
    return response.json()

page_content = get_notion_page_content(PAGE_ID)


In [5]:
page_content

2. 將 DataFrame 合併文本和公式並保留格式

In [6]:
def blocks_to_dataframe(blocks):
    data = []
    for block in blocks['results']:
        block_type = block['type']
        content = ''
        
        # 處理有 rich_text 的區塊
        if 'rich_text' in block.get(block_type, {}):
            for item in block[block_type]['rich_text']:
                if item['type'] == 'text':
                    content += item['text']['content']
                elif item['type'] == 'equation':
                    content += f"$$ {item['equation']['expression']} $$"
        
        data.append({'id': block['id'], 'type': block_type, 'content': content})
    
    return pd.DataFrame(data)

df = blocks_to_dataframe(page_content)


3. 提取並合併文字和公式（不換行）

In [7]:
def format_content_for_notion(content):
    parts = content.split("$$")
    formatted_parts = []
    
    for i, part in enumerate(parts):
        part = part.strip()
        if i % 2 == 1:  # 偶數部分是公式
            formatted_parts.append({
                "type": "equation",
                "equation": {"expression": part}
            })
        else:
            if part:  # 將文字前後加上空白
                formatted_parts.append({
                    "type": "text",
                    "text": {"content": f" {part} "}  # 在文字前後加空白
                })
    
    return formatted_parts

def combine_text_and_equations(df):
    combined_blocks = []
    for _, row in df.iterrows():
        content = row['content']
        notion_block_content = format_content_for_notion(content)
        
        combined_blocks.append({
            'type': 'paragraph',
            'paragraph': {
                'rich_text': notion_block_content
            }
        })

    return combined_blocks

combined_data = combine_text_and_equations(df)

4. 將合併後的資料上傳到 Notion 頁面

In [8]:
def upload_to_notion(page_id, combined_blocks):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    children_blocks = combined_blocks
    payload = {
        "children": children_blocks
    }
    response = requests.patch(url, json=payload, headers=HEADERS)
    return response.json()

upload_to_notion(PAGE_ID, combined_data)